# Necessary Weather Data

In this Jupyter notebook we note the necessary variables needed for PV production calculations. Moreover, we are going to list a series of possible weather databases.

## Weather databases

1. Meteonorm 8.1
2. PVGIS TMY
3. SOLCast TMY
4. Solar Anywhere TGY
5. Solargis TMY
6. NASA-SSE

## Weather variables to take into account

1. Global Horizontal Irradiance (GHI - kWh/m^2/month)
2. Horizontal Diffuse Radiation (DHI - kWh/m^2/month)
3. Temperature (celcius)
4. Wind velocity (m/s)
5. Linke turbidity (-)
6. Relative Humidity (%)

###### Side note on Linke Turbidity (TL)

Linke turbidity (TL) is really a lumped “haziness” index that accounts for all the extra extinction of sunlight beyond what a perfectly clean, dry atmosphere would do. That “extra extinction” comes mainly from two things:

Aerosols (pollution, dust, smoke, volcanic particles, etc.)
This is the “pollution” part. Urban smog or Saharan dust events push TL upward.

Water vapor absorption
Humid tropical air or summer days with high precipitable water also increase TL.

TL isn’t only a pollution indicator. A very humid but clean region (say, the Amazon) will also have a high TL even if the air is not polluted.

In [1]:
# Fetch hourly PVGIS SARAH-3 with pvlib (multi-year)
# pip install pvlib pandas pyarrow  (pyarrow optional for Parquet)

from __future__ import annotations
import datetime
import pathlib, time
import pandas as pd
import pvlib

LAT, LON = 41.9028, 12.4964
START_YEAR = 2011       
N_YEARS    = 12                
RADDATABASE = "PVGIS-SARAH3"
API_BASE = "https://re.jrc.ec.europa.eu/api/v5_3/"
OUTDIR = pathlib.Path("pvgis_sarah3_hourly"); OUTDIR.mkdir(exist_ok=True)
TILT = 0
AZIMUTH = 0

def fetch_year(year: int) -> pd.DataFrame:
    """Return hourly PVGIS data (UTC index) with pvlib-mapped column names."""
    start, end = f"{year}-01-01", f"{year}-12-31"
    df, meta = pvlib.iotools.get_pvgis_hourly(
        LAT, LON,
        start=start, end=end,
        raddatabase=RADDATABASE,
        components=True,               # get GHI/DHI/DNI etc.
        surface_tilt=TILT, 
        surface_azimuth=AZIMUTH,  # horizontal
        usehorizon=True,
        map_variables=True,
        url=API_BASE,
        outputformat="json",
    )
    return df.sort_index()


all_years = []
for i in range(N_YEARS):
    y = START_YEAR + i
    print(f"Fetching {y} ({RADDATABASE}) …")
    for attempt in range(1, 4):
        try:
            dfx = fetch_year(y)
            dfx['TL'] = pvlib.clearsky.lookup_linke_turbidity(dfx.index, LAT, LON).values
            if dfx.index.tz is None or dfx.index.tz == datetime.timezone.utc:
                dfx.index = dfx.index.tz_convert("Europe/Rome")
            all_years.append(dfx)
            break
        except Exception as e:
            if attempt == 3:
                raise
            time.sleep(1.5 * attempt)



Fetching 2011 (PVGIS-SARAH3) …
Fetching 2012 (PVGIS-SARAH3) …
Fetching 2013 (PVGIS-SARAH3) …
Fetching 2014 (PVGIS-SARAH3) …
Fetching 2015 (PVGIS-SARAH3) …
Fetching 2016 (PVGIS-SARAH3) …
Fetching 2017 (PVGIS-SARAH3) …
Fetching 2018 (PVGIS-SARAH3) …
Fetching 2019 (PVGIS-SARAH3) …
Fetching 2020 (PVGIS-SARAH3) …
Fetching 2021 (PVGIS-SARAH3) …
Fetching 2022 (PVGIS-SARAH3) …


In [7]:
weather = pd.concat(all_years).sort_index()
weather.head()

,poa_direct,poa_sky_diffuse,poa_ground_diffuse,solar_elevation,temp_air,wind_speed,Int,TL
time,,,,,,,,
2011-01-01 01:10:00+01:00,0.0,0.0,0.0,0.0,5.88,1.52,0,2.792742
2011-01-01 02:10:00+01:00,0.0,0.0,0.0,0.0,5.55,1.66,0,2.792742
2011-01-01 03:10:00+01:00,0.0,0.0,0.0,0.0,5.36,1.66,0,2.792742
2011-01-01 04:10:00+01:00,0.0,0.0,0.0,0.0,4.88,1.79,0,2.792742
2011-01-01 05:10:00+01:00,0.0,0.0,0.0,0.0,4.64,1.72,0,2.792742


# Power Output Calculation

In [ ]:
# --- CONFIG: choose whether to reuse PVGIS POA or recompute with pvlib ---
USE_PVGIS_POA = True   # A: reuse PVGIS 'poa_*' columns
# USE_PVGIS_POA = False  # B: recompute POA from GHI/DHI/DNI with Perez

import numpy as np
import pandas as pd
import pvlib

# --- build POA exactly as ModelChain expects (no transposition) ---
loc = pvlib.location.Location(LAT, LON, tz="Europe/Rome")
solpos = loc.get_solarposition(df.index)
rev = pvlib.irradiance.gti_dirint(
    poa_global=weather['poa_direct']+weather['poa_sky_diffuse']+weather['poa_ground_diffuse'],
    aoi=pvlib.irradiance.aoi(TILT, AZIMUTH, solpos['apparent_zenith'], solpos['azimuth']),
    solar_zenith=solpos['apparent_zenith'],
    solar_azimuth=solpos['azimuth'],
    times=weather.index,
    surface_tilt=TILT,
    surface_azimuth=AZIMUTH,
)
weather[['ghi','dni','dhi']] = rev[['ghi','dni','dhi']]

# --- pick module & inverter as you already do ---
cecm = pvlib.pvsystem.retrieve_sam('cecmod')
ceci = pvlib.pvsystem.retrieve_sam('cecinverter')
module = cecm[cecm.columns[0]]
inverter = ceci[ceci.columns[0]]

# --- PVSyst temperature model parameters (choose mounting) ---
# Open rack defaults (PVSyst-style). Change to (20,6) for rooftop if needed.
U_C, U_V = 29.0, 0.0

eta_m = float(module['I_mp_ref']*module['V_mp_ref']) / (1000.0 * float(module['A_c']))

temperature_model_parameters = {
    'u_c': U_C,
    'u_v': U_V,
    'eta_m': eta_m,
    'alpha_absorption': 0.9,
}

system = pvlib.pvsystem.PVSystem(
    surface_tilt=TILT,
    surface_azimuth=AZIMUTH,
    module_parameters=module,
    inverter_parameters=inverter,
    temperature_model_parameters=temperature_model_parameters,
    albedo=0.2,
)

mc = pvlib.modelchain.ModelChain(
    system, loc,
    # transposition_model won't be used because we supply POA directly
    aoi_model='physical',
    spectral_model='no_loss',
    losses_model='pvwatts',
    temperature_model='pvsyst',   # << use PVSyst-style temps
)

# --- Weather frame with POA (no GHI/DHI/DNI needed) ---
weather = df[['temp_air', 'wind_speed', 'poa_global', 'poa_direct', 'poa_diffuse']].copy()

# --- Run: MC will use your POA as-is and skip transposition ---
mc.run_model(weather)

ac = mc.results.ac.rename('P_ac_W')



KeyError: 'poa_global'